<a href="https://colab.research.google.com/github/satyabratkumarsingh/option-portfolio-encoder-decoder/blob/main/Advanced_Set_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install comet_ml
!pip install tqdm
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
def delete_file_from_drive(full_file_path):
  if os.path.exists(full_file_path):
      try:
          os.remove(full_file_path)
          print(f"File '{full_file_path}' successfully deleted from Google Drive.")
      except Exception as e:
          print(f"Error deleting file '{full_file_path}': {e}")
  else:
      print(f"File '{full_file_path}' not found at '{full_file_path}'.")


In [ ]:
import random
import numpy as np
import torch
import itertools
from itertools import product
from torch.utils.data import Dataset, DataLoader
import gc # For garbage collection

# Parameters
MU = 0.05
SIGMA = 0.2
T = 1.0 # Time to maturity
NOISE_STD = 0.005
MIN_PRICE_RANGE = 100
MAX_PRICE_RANGE = 200

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def generate_option_prices_for_idx(idx, n, weights=None):
    # Use numpy's default_rng for better random number generation and seeding
    rng = np.random.default_rng(idx)
    # Use torch.manual_seed for PyTorch operations
    torch.manual_seed(idx)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(idx)

    # Generate S_0
    random_number = rng.integers(MIN_PRICE_RANGE, MAX_PRICE_RANGE + 1) # +1 because randint is inclusive
    min_price = random_number
    max_price = random_number + 5
    S_0 = rng.uniform(min_price, max_price)

    # Generate option types
    option_types = rng.choice(["call", "put"], size=n)
    option_types_numeric = np.where(option_types == "call", 1, 0).astype(np.float32) # Ensure float32

    # Generate X_prices (strike prices)
    K_prices = np.zeros(n, dtype=np.float32) # Ensure float32
    for i in range(n):
        K_prices[i] = S_0 * rng.uniform(0.90, 1.20)

    # Generate or use weights
    if weights is None:
        # If weights are not provided, generate them using generate_combinatorial_weights_manageable
        weight_sets = generate_combinatorial_weights_manageable(n)
        weights_array = weight_sets[0]  # Use the first (and only) set of weights
    else:
        weights_array = np.array(weights, dtype=np.float32)  # Ensure float32

    return K_prices, option_types_numeric, S_0, weights_array


def generate_combinatorial_weights_manageable(n, base_weights=[-0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75]):
    weight_sets = []

    # Handle the case where n < 2
    if n < 2:
        weights = np.zeros(n, dtype=np.float32)
        if n == 1:
            # If only one position, assign a long position (1.0)
            weights[0] = 1.0
        weight_sets.append(weights)
        return weight_sets

    # Generate a single portfolio: either one long or one short, and the rest from combinatorics
    weights = np.zeros(n, dtype=np.float32)

    # Randomly choose if we want a long or short portfolio
    is_long = random.choice([True, False])

    if is_long:
        # Choose one position to be long (1.0)
        long_idx = random.randint(0, n - 1)
        weights[long_idx] = 1.0
    else:
        # Choose one position to be short (-1.0)
        short_idx = random.randint(0, n - 1)
        weights[short_idx] = -1.0

    # Fill remaining positions with combinatorial weights from base_weights
    remaining_positions = [i for i in range(n) if weights[i] == 0]  # Find positions not yet filled
    combinatorics = np.random.choice(base_weights, size=len(remaining_positions), replace=True)

    # Assign combinatorial weights to the remaining positions without normalization
    weights[remaining_positions] = combinatorics

    weight_sets.append(weights)

    return weight_sets



import torch

def black_scholes_delta(S, K, T, r, sigma, option_type):
    """
    Computes the Black-Scholes delta for a call or put option.

    Args:
        S (Tensor): Spot price [any shape]
        K (Tensor): Strike price [same shape as S or broadcastable]
        T (float or Tensor): Time to maturity (scalar or broadcastable)
        r (float or Tensor): Risk-free rate (scalar or broadcastable)
        sigma (float or Tensor): Volatility (scalar or broadcastable)
        option_type (Tensor): 1 for call, 0 for put [same shape as S]

    Returns:
        Tensor: Delta of the option [same shape as S]
    """
    eps = 1e-8  # Numerical stability for sqrt
    device = S.device

    T = torch.as_tensor(T, device=device, dtype=S.dtype)
    r = torch.as_tensor(r, device=device, dtype=S.dtype)
    sigma = torch.as_tensor(sigma, device=device, dtype=S.dtype)

    d1 = (torch.log(S / K + eps) + (r + 0.5 * sigma ** 2) * T) / (sigma * torch.sqrt(T + eps))

    # More stable, recommended: torch.special.ndtr(d1) (if available)
    N_d1 = torch.distributions.Normal(0.0, 1.0).cdf(d1)

    delta = torch.where(option_type == 1, N_d1, N_d1 - 1.0)
    return delta

def compute_cashflow(portfolio, S_T, weights):
    strikes = portfolio[..., 0]
    types = portfolio[..., 1]
    weights = weights.to(S_T.device)

    # Compute payoff-based cashflow as before
    payoffs = torch.where(
        types == 1,
        torch.relu(S_T - strikes),
        torch.relu(strikes - S_T)
    )
    weighted_payoffs = payoffs * weights
    cashflow = weighted_payoffs.sum(dim=-1, keepdim=True)

   # --- Continuous Derivative (Black-Scholes Delta) ---
    delta = black_scholes_delta(
        S_T, strikes, T=T, r=MU, sigma=SIGMA, option_type=types
    )
    weighted_delta = weights * delta
    derivative = weighted_delta.sum(dim=-1, keepdim=True)

    return cashflow.to(torch.float32), derivative.to(torch.float32)



In [ ]:

import os
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler


class OperatorDatasetStandarization(Dataset):

    def __init__(self, num_samples, portfolio_size, num_samples_S_T,
                 K_Scalar=None, S_T_scalar=None, cashflow_scaler=None,
                 is_fitting_mode=False):

        self.num_samples = num_samples
        self.portfolio_size = portfolio_size
        self.num_samples_S_T = num_samples_S_T
        self.is_fitting_mode = is_fitting_mode # Store the mode

        # Load or generate K_Scalar and S_T_scalar
        if not self.is_fitting_mode:
          if K_Scalar is None or S_T_scalar is None or cashflow_scaler is None:
              raise ValueError("K_Scalar, S_T_scalar, and cashflow_scaler must be explicitly provided.")
          else:
            self.K_Scalar = K_Scalar
            self.S_T_scalar = S_T_scalar
            self.cashflow_scaler = cashflow_scaler
        else:
            self.K_Scalar = None
            self.S_T_scalar = None
            self.cashflow_scaler = None


    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return self.num_samples

    def __getitem__(self, idx):
        """
        Generates and returns a single data sample (portfolio, S_T, cashflow, derivative).
        This method is called by the DataLoader.
        """

        K, option_types, S_0, weights = generate_option_prices_for_idx(
            idx, self.portfolio_size
        )

        portfolio_features_tensor = torch.stack([
            torch.tensor(K, dtype=torch.float32, device=DEVICE),
            torch.tensor(option_types, dtype=torch.float32, device=DEVICE),
            torch.tensor(weights, dtype=torch.float32, device=DEVICE)
        ], dim=-1)

        weights_i = torch.tensor(weights, dtype=torch.float32, device=DEVICE)
        K_i = torch.tensor(K, dtype=torch.float32, device=DEVICE)
        S_0_i = torch.tensor(S_0, dtype=torch.float32, device=DEVICE)
        Z = torch.clamp(torch.randn(self.num_samples_S_T, device=DEVICE), -3, 3)

        S_T_i = S_0_i * torch.exp((MU - 0.5 * SIGMA**2) * T + SIGMA * torch.sqrt(torch.tensor(T, device=DEVICE)) * Z)
        S_T_i += torch.randn_like(S_T_i, device=DEVICE) * (NOISE_STD * S_T_i)

        # Normalization for K and S_T (always apply if scalers are present and not in fitting mode)
        # Note: In fitting mode, K and S_T are still generated, but their *normalized* versions
        # are not what we're collecting for the cashflow scaler.
        if not self.is_fitting_mode:
            K_i_cpu = K_i.reshape(-1, 1).cpu()
            K_i_normalized = torch.tensor(self.K_Scalar.transform(K_i_cpu), dtype=torch.float32, device=DEVICE)
            S_T_i_normalized = torch.from_numpy(self.S_T_scalar.transform(S_T_i.cpu().numpy().reshape(-1, 1))).to(DEVICE).squeeze()
        else: # In fitting mode, just use raw K_i and S_T_i for generating raw cashflow
            K_i_normalized = K_i # This won't be used for input features directly, but kept for clarity
            S_T_i_normalized = S_T_i


        # Compute cashflow and derivative (ALWAYS raw when generated in __getitem__)
        cashflow_i_raw, derivative_i_raw = compute_cashflow(
            portfolio_features_tensor.expand(self.num_samples_S_T, -1, -1),
            S_T_i.unsqueeze(-1),
            weights_i.expand(self.num_samples_S_T, -1)
        )

        # ===== CASHFLOW NORMALIZATION (Apply only if a scaler is provided and not in fitting mode) =====
        if self.cashflow_scaler is not None and not self.is_fitting_mode:
            # Convert to numpy on CPU for scaler, then back to tensor
            cashflow_i_normalized_np = self.cashflow_scaler.transform(cashflow_i_raw.cpu().numpy().reshape(-1, 1))
            cashflow_i_to_return = torch.from_numpy(cashflow_i_normalized_np).to(DEVICE).squeeze()
        else:
            # If no scaler or in fitting mode, return the raw cashflow
            cashflow_i_to_return = cashflow_i_raw.squeeze()

        # Update portfolio_i_normalized with normalized K_i if not in fitting mode
        if not self.is_fitting_mode:
            portfolio_i_normalized = portfolio_features_tensor.clone()
            portfolio_i_normalized[:, 0] = K_i_normalized.squeeze()
        else: # In fitting mode, just return original K in portfolio_features_tensor for now
            portfolio_i_normalized = portfolio_features_tensor.clone()


        return portfolio_i_normalized, S_T_i_normalized,  cashflow_i_to_return, derivative_i_raw.squeeze()


In [ ]:
# Fit ST and K

from sklearn.preprocessing import StandardScaler

DRIVE_PATH = "/content/drive/MyDrive/Ucl/"
K_SCALAR_FILE = os.path.join(DRIVE_PATH, 'K_Scalar_Advanced.pkl')
ST_SCALAR_FILE = os.path.join(DRIVE_PATH, 'S_T_Scalar_Advanced.pkl')
CASHFLOW_SCALAR_FILE = os.path.join(DRIVE_PATH, 'Cashflow_Scalar_Advanced.pkl')



def fit_K_ST_scalers(train_loader, save_path_K=K_SCALAR_FILE, save_path_ST=ST_SCALAR_FILE):
    print("Fitting K and S_T scalers from training set...")
    all_K = []
    all_S_T = []

    for batch in tqdm(train_loader, desc="Collecting K and S_T for scalers"):
        portfolio_real, s_t_real, _, _ = batch

        # K is the first column of the portfolio features
        K_real = portfolio_real[:, :, 0].cpu().numpy().reshape(-1, 1)
        S_T_real = s_t_real.cpu().numpy().reshape(-1, 1)

        all_K.append(K_real)
        all_S_T.append(S_T_real)

    K_all_np = np.concatenate(all_K, axis=0)
    S_T_all_np = np.concatenate(all_S_T, axis=0)

    K_scalar = StandardScaler()
    K_scalar.fit(K_all_np)
    joblib.dump(K_scalar, save_path_K)

    S_T_scalar = StandardScaler()
    S_T_scalar.fit(S_T_all_np)
    joblib.dump(S_T_scalar, save_path_ST)

    print(f"✅ Saved K scalar to: {save_path_K}")
    print(f"✅ Saved S_T scalar to: {save_path_ST}")
    print(f"K mean: {K_scalar.mean_[0]:.4f}, std: {K_scalar.scale_[0]:.4f}")
    print(f"S_T mean: {S_T_scalar.mean_[0]:.4f}, std: {S_T_scalar.scale_[0]:.4f}")

    return K_scalar, S_T_scalar

def fit_cashflow_scaler(train_loader, save_path=CASHFLOW_SCALAR_FILE):
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import joblib

    all_cashflows = []

    for portfolio, s_t, cashflow, _ in tqdm(train_loader, desc="Fitting Cashflow Scaler"):
        cashflow = cashflow.detach().cpu().numpy().reshape(-1, 1)
        all_cashflows.append(cashflow)

    cashflows_np = np.concatenate(all_cashflows, axis=0)

    scaler = StandardScaler()
    scaler.fit(cashflows_np)
    joblib.dump(scaler, save_path)

    print(f"✅ Saved Cashflow Scaler to: {save_path}")
    print(f"Cashflow Mean: {scaler.mean_[0]:.4f}, Std Dev: {scaler.scale_[0]:.4f}")

    return scaler



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torch.amp import autocast, GradScaler
from tqdm import tqdm
import math

FEED_FWD_DEPTH = 4


class TrunkNet(nn.Module):
    def __init__(self, input_dim=1, latent_dim=64, hidden_dim=128,
                 num_layers=6, dropout_prob=0.1):
        super(TrunkNet, self).__init__()

        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        # Residual blocks
        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout_prob)
            ) for _ in range(num_layers)
        ])

        # Output projection
        self.output_proj = nn.Linear(hidden_dim, latent_dim)

    def forward(self, S_T):
        if S_T.dim() == 1:
            S_T = S_T.unsqueeze(-1)
        elif S_T.dim() == 2:
            S_T = S_T.unsqueeze(-1)

        x = self.input_proj(S_T)

        # Residual connections
        for block in self.blocks:
            x = x + block(x)

        return self.output_proj(x)


class ISAB(nn.Module):
    """Induced Set Attention Block using PyTorch's MultiheadAttention"""
    def __init__(self, d_model, num_heads, num_inds, dropout=0.1):
        super().__init__()
        self.num_inds = num_inds
        self.inducing_points = nn.Parameter(torch.randn(num_inds, d_model))

        # Use PyTorch's built-in MultiheadAttention
        self.attention1 = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.attention2 = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)

        self.ffn1 = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )

        self.ffn2 = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        batch_size = x.size(0)

        # Expand inducing points for batch
        I = self.inducing_points.unsqueeze(0).expand(batch_size, -1, -1)

        # First attention: I attends to X
        attn_out1, _ = self.attention1(I, x, x)
        I = self.norm1(I + attn_out1)

        # Feed-forward on inducing points
        ffn_out1 = self.ffn1(I)
        I = self.norm2(I + ffn_out1)

        # Second attention: X attends to I
        attn_out2, _ = self.attention2(x, I, I)
        x = self.norm3(x + attn_out2)

        # Feed-forward on original input
        ffn_out2 = self.ffn2(x)
        x = self.norm4(x + ffn_out2)

        return x


class PMA(nn.Module):
    """Pooling by Multihead Attention using PyTorch's MultiheadAttention"""
    def __init__(self, d_model, num_heads, num_seeds, dropout=0.1):
        super().__init__()
        self.num_seeds = num_seeds
        self.seed_vectors = nn.Parameter(torch.randn(num_seeds, d_model))

        # Use PyTorch's built-in MultiheadAttention
        self.attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        batch_size = x.size(0)

        # Expand seed vectors for batch
        S = self.seed_vectors.unsqueeze(0).expand(batch_size, -1, -1)

        # Attention: seeds attend to input
        attn_out, _ = self.attention(S, x, x)
        S = self.norm1(S + attn_out)

        # Feed-forward
        ffn_out = self.ffn(S)
        S = self.norm2(S + ffn_out)

        return S


class SAB(nn.Module):
    """Set Attention Block using PyTorch's TransformerEncoderLayer"""
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_model * FEED_FWD_DEPTH,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )

    def forward(self, x):
        return self.transformer_layer(x)


class EnhancedSetTransformerEncoder(nn.Module):
    """Set Transformer with ISAB and PMA for better generalization"""
    def __init__(self, portfolio_feature_dim=3, latent_dim=128, hidden_dim=64,
                 num_heads=2, dropout_prob=0.1, num_inds=32, num_seeds=1,
                 use_isab=True, num_layers=2):
        super().__init__()

        # Ensure hidden_dim is divisible by num_heads
        if hidden_dim % num_heads != 0:
            hidden_dim = ((hidden_dim // num_heads) + 1) * num_heads

        self.hidden_dim = hidden_dim
        self.use_isab = use_isab

        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(portfolio_feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        # Encoder layers
        self.encoder_layers = nn.ModuleList()
        for _ in range(num_layers):
            if use_isab:
                self.encoder_layers.append(
                    ISAB(hidden_dim, num_heads, num_inds, dropout_prob)
                )
            else:
                self.encoder_layers.append(
                    SAB(hidden_dim, num_heads, dropout_prob)
                )

        # Pooling layer
        self.pooling = PMA(hidden_dim, num_heads, num_seeds, dropout_prob)

        # Output projection
        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, portfolio):
        """
        Args:
            portfolio: [batch_size, portfolio_size, portfolio_feature_dim]
        Returns:
            [batch_size, latent_dim]
        """
        # Input projection
        x = self.input_proj(portfolio)  # [B, P, H]

        # Encoder layers
        for layer in self.encoder_layers:
            x = layer(x)

        # Pooling
        x = self.pooling(x)  # [B, num_seeds, H]

        # If we have multiple seeds, we can pool them further
        if x.size(1) > 1:
            x = x.mean(dim=1)  # [B, H]
        else:
            x = x.squeeze(1)  # [B, H]

        # Final projection
        out = self.output_proj(x)  # [B, latent_dim]

        return out


class OptimizedSetTransformerEncoder(nn.Module):
    """Original Set Transformer using PyTorch's built-in TransformerEncoder"""
    def __init__(self, portfolio_feature_dim=3, latent_dim=128, hidden_dim=64,
                 num_layers=1, num_heads=2, dropout_prob=0.1):
        super().__init__()

        # Ensure hidden_dim is divisible by num_heads
        if hidden_dim % num_heads != 0:
            hidden_dim = ((hidden_dim // num_heads) + 1) * num_heads

        self.hidden_dim = hidden_dim

        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(portfolio_feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        # PyTorch's built-in TransformerEncoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * FEED_FWD_DEPTH,
            dropout=dropout_prob,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
            enable_nested_tensor=False
        )

        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, portfolio):
        """
        Args:
            portfolio: [batch_size, portfolio_size, portfolio_feature_dim]
        Returns:
            [batch_size, latent_dim]
        """
        x = self.input_proj(portfolio)  # [B, P, H]

        # PyTorch transformer expects [batch, seq, feature] with batch_first=True
        x = self.transformer_encoder(x)  # [B, P, H]

        pooled = x.mean(dim=1) + x.max(dim=1).values

        # Final projection
        out = self.output_proj(pooled)  # [B, latent_dim]

        return out


class OptimizedDeepONet(nn.Module):
    """DeepONet with choice of Set Transformer architectures"""
    def __init__(self, portfolio_feature_dim=3, hidden_dim=64, latent_dim=128,
                 dropout_prob=0.1, num_heads=2, use_enhanced_transformer=True,
                 num_inds=32, num_seeds=1):
        super().__init__()

        if hidden_dim % num_heads != 0:
            recommended = ((hidden_dim // num_heads) + 1) * num_heads
            raise ValueError(
                f"hidden_dim ({hidden_dim}) must be divisible by num_heads ({num_heads}). "
                f"Try hidden_dim={recommended}"
            )

        self.latent_dim = latent_dim

        # Choose branch network architecture
        if use_enhanced_transformer:
            self.branch_net = EnhancedSetTransformerEncoder(
                portfolio_feature_dim=portfolio_feature_dim,
                latent_dim=latent_dim,
                hidden_dim=hidden_dim,
                num_heads=num_heads,
                dropout_prob=dropout_prob,
                num_inds=num_inds,
                num_seeds=num_seeds,
                use_isab=True,
                num_layers=2
            )
        else:
            self.branch_net = OptimizedSetTransformerEncoder(
                portfolio_feature_dim=portfolio_feature_dim,
                latent_dim=latent_dim,
                hidden_dim=hidden_dim,
                dropout_prob=dropout_prob,
                num_heads=num_heads
            )

        # Trunk network
        self.trunk_net = TrunkNet(
            input_dim=1,
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            dropout_prob=dropout_prob
        )

        # DeepONet parameters
        self.bias = nn.Parameter(torch.zeros(1))
        self.branch_scale = nn.Parameter(torch.ones(1) * 0.8)
        self.trunk_scale = nn.Parameter(torch.ones(1) * 0.8)

    def forward(self, portfolio, S_T):
        """
        Args:
            portfolio: [batch_size, portfolio_size, 3] - portfolio features
            S_T: [batch_size, num_S_T_samples] - multiple S_T values per portfolio

        Returns:
            cashflows: [batch_size, num_S_T_samples] - predicted cashflows for each S_T
        """
        # Branch network: encode portfolio
        branch_out = self.branch_net(portfolio)  # [batch_size, latent_dim]
        branch_out = branch_out * self.branch_scale

        # Trunk network: process S_T values
        trunk_out = self.trunk_net(S_T)  # [batch_size, num_S_T_samples, latent_dim]
        trunk_out = trunk_out * self.trunk_scale

        # Compute dot product: branch ⊗ trunk
        branch_expanded = branch_out.unsqueeze(1)  # [batch_size, 1, latent_dim]

        interaction = (branch_expanded * trunk_out).sum(dim=-1)  # [batch_size, num_S_T_samples]

        # Add bias
        cashflows = interaction
        return cashflows

In [ ]:
def compute_gradient_stats(model):
    """Compute gradient statistics - fixed for single-element tensors"""
    total_norm = 0
    param_count = 0
    gradient_stats = {}

    for name, param in model.named_parameters():
        if param.grad is not None:
            param_norm = param.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
            param_count += 1

            grad_data = param.grad.data
            if grad_data.numel() > 1:
                grad_std = grad_data.std(unbiased=False).item()
            else:
                grad_std = 0.0 # Handle single-element tensors

            gradient_stats[name] = {
                'norm': param_norm.item(),
                'mean': grad_data.mean().item(),
                'std': grad_std,
                'max': grad_data.max().item(),
                'min': grad_data.min().item(),
                'shape': list(param.grad.shape),
                'numel': grad_data.numel()
            }

    total_norm = total_norm ** (1. / 2)
    return total_norm, gradient_stats, param_count

def print_gradient_summary(gradient_stats, total_norm, epoch, batch_idx=None):
    """Enhanced gradient summary with more details"""
    prefix = f"Epoch {epoch}" + (f", Batch {batch_idx}" if batch_idx is not None else "")
    print(f"\n🔍 === Gradient Analysis - {prefix} ===")
    print(f"Total Gradient Norm: {total_norm:.6f}")

    if total_norm > 30.0:
        print("🚨 CRITICAL: Severe gradient explosion! Consider stopping training.")
    elif total_norm > 20.0:
        print("⚠️  SEVERE: Major gradient explosion detected!")
    elif total_norm > 10.0:
        print("⚠️  WARNING: Moderate gradient explosion detected!")
    elif total_norm < 1e-6:
        print("⚠️  WARNING: Vanishing gradients detected!")
    else:
        print("✅ Gradient norm is healthy")

    sorted_layers = sorted(gradient_stats.items(), key=lambda x: x[1]['norm'], reverse=True)
    print(f"\nTop 5 layers by gradient norm (out of {len(gradient_stats)} total):")
    for i, (layer_name, stats) in enumerate(sorted_layers[:5]):
        status = "🔥" if stats['norm'] > 3.0 else "⚠️" if stats['norm'] > 1.0 else "✅"
        print(f"  {status} {i+1}. {layer_name}: {stats['norm']:.4f}")
        print(f"      Shape: {stats['shape']}, Elements: {stats['numel']}")
        print(f"      Mean: {stats['mean']:.6f}, Std: {stats['std']:.6f}")

    print("=" * 60)

In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler



def rescale_derivative_autograd(pred_derivative_from_normalized_input, S_T_scalar_normalizer):

    if S_T_scalar_normalizer is None:
        # If no scaler was used, the derivative is already on the correct scale
        return pred_derivative_from_normalized_input

    # Ensure the scale factor is a torch tensor on the correct device
    device = pred_derivative_from_normalized_input.device
    dtype = pred_derivative_from_normalized_input.dtype

    if hasattr(S_T_scalar_normalizer, 'data_max_') and hasattr(S_T_scalar_normalizer, 'data_min_'):
        # Case: MinMaxScaler
        # X_normalized = (X_original - min) / (max - min)
        # d(X_normalized)/d(X_original) = 1 / (max - min)
        # So, d(L)/d(X_original) = d(L)/d(X_normalized) * (1 / (max - min))
        # Thus, d(L)/d(X_original) = d(L)/d(X_normalized) / (max - min)
        # Or equivalently, d(L)/d(X_original) = d(L)/d(X_normalized) * (max - min) (your current implementation)
        # This part of your code for MinMaxScaler is correct.

        # Ensure data_range is a tensor
        data_range = torch.tensor(S_T_scalar_normalizer.data_max_ - S_T_scalar_normalizer.data_min_,
                                  dtype=dtype, device=device).squeeze() # Squeeze if it's (1,)

        if data_range.item() < 1e-6: # Use .item() for scalar comparison
            print(f"⚠️ WARNING: S_T data_range is extremely small ({data_range.item():.4e}). Derivative may be unstable.")
            return torch.zeros_like(pred_derivative_from_normalized_input)

        # MinMaxScaler: derivative must be multiplied by the range
        return pred_derivative_from_normalized_input * data_range

    else:
        raise ValueError(f"Unsupported scaler type: {type(S_T_scalar_normalizer)}. Expected StandardScaler.")


class ExtendedEarlyStopping:
    # ... (no changes needed here) ...
    def __init__(self, patience=30, min_delta=0.0005, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.wait = 0
        self.stopped_epoch = 0
        self.best = float('inf')
        self.best_weights = None

    def __call__(self, val_loss, model=None):
        if val_loss < self.best - self.min_delta:
            self.best = val_loss
            self.wait = 0
            if model is not None and self.restore_best_weights:
                self.best_weights = model.state_dict().copy()
        else:
            self.wait += 1

        if self.wait >= self.patience:
            self.stopped_epoch = True
            if model is not None and self.restore_best_weights and self.best_weights is not None:
                model.load_state_dict(self.best_weights)

        return self.stopped_epoch


In [ ]:
def get_stable_hyperparameters():
    """Return more stable hyperparameters"""
    return {
        "learning_rate": 1e-4,
        "weight_decay": 1e-5 ,
        "lambda_deriv": 0.2,
        "lambda_reg": 1e-4,
        "gradient_clip_norm": 5,
        "batch_size": 128,
        "scheduler_T0": 200,
        "early_stopping_patience": 50,
    }


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.amp import autocast, GradScaler

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.amp import autocast, GradScaler


class OptimizedTrainer:
    def __init__(self, model, device='cuda', monitor_gradients=True,
                 scale_warmup_epochs=5, initial_scale=0.05, final_scale=1.0,
                 learning_rate=5e-6, lambda_deriv_weight=0.1, weight_decay=1e-4, K_Scalar = None, ST_Scalar=None): # ADDED: learning_rate, lambda_deriv_weight, weight_decay for consistency
        self.model = model.to(device)
        self.device = device
        self.monitor_gradients = monitor_gradients
        self.lambda_deriv_weight = lambda_deriv_weight # Store this for compute_loss

        self.optimizer = optim.AdamW(
            model.parameters(),
            lr=learning_rate, # Use the passed learning_rate
            weight_decay=weight_decay, # Use the passed weight_decay (or make it a constant here if not configurable)
            betas=(0.9, 0.999),
            eps=1e-8
        )

        self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=100, T_mult=2, eta_min=1e-6 # T_0 might also be a hyperparameter
        )

        self.scaler = GradScaler()
        self.K_scalar = K_Scalar
        self.S_T_scalar = ST_Scalar


        self.cashflow_scaler = joblib.load(CASHFLOW_SCALAR_FILE) if os.path.exists(CASHFLOW_SCALAR_FILE) else None
        if self.cashflow_scaler:
          print(f"LOADED CASHFLOW SCALARD FROM {CASHFLOW_SCALAR_FILE}")

        self.mse_loss = nn.MSELoss()
        self.huber_loss = nn.SmoothL1Loss(beta=1.0)


        # Scale warmup parameters
        self.scale_warmup_epochs = scale_warmup_epochs
        self.initial_scale = initial_scale
        self.final_scale = final_scale

        # Initialize model scales
        if hasattr(self.model, 'branch_scale') and hasattr(self.model, 'trunk_scale'):
            with torch.no_grad():
                self.model.branch_scale.fill_(initial_scale)
                self.model.trunk_scale.fill_(initial_scale)

    def check_model_health(self, epoch, batch_idx):
        """Check model parameters for NaN/Inf values"""
        for name, param in self.model.named_parameters():
            if torch.isnan(param).any():
                print(f"🚨 NaN parameter found: {name} at Epoch {epoch}, Batch {batch_idx}")
                return False
            if torch.isinf(param).any():
                print(f"🚨 Inf parameter found: {name} at Epoch {epoch}, Batch {batch_idx}")
                return False
        return True


    def compute_loss(self, pred_cashflow, true_cashflow, pred_deriv, true_deriv, lambda_reg=1e-4):


      if self.cashflow_scaler is not None:

          # pred_cashflow_rescaled = rescale_cashflow(pred_cashflow, self.cashflow_scaler)
          # true_cashflow_rescaled = rescale_cashflow(true_cashflow, self.cashflow_scaler)
          # cashflow_loss = self.huber_loss(pred_cashflow_rescaled, true_cashflow_rescaled)
          cashflow_loss = self.huber_loss(pred_cashflow, true_cashflow)
      else:
          cashflow_loss = self.huber_loss(pred_cashflow, true_cashflow)
      if true_deriv is not None:
          pred_deriv_rescaled = rescale_derivative_autograd(pred_deriv, self.S_T_scalar)
          deriv_loss = self.huber_loss(pred_deriv_rescaled, true_deriv)
          total_loss = cashflow_loss + self.lambda_deriv_weight * deriv_loss
      else:
          deriv_loss = torch.tensor(0.0, device=pred_cashflow.device)
          total_loss = cashflow_loss

      return total_loss, cashflow_loss, deriv_loss


    def train_step(self, portfolio, S_T, cashflow, true_derivative,
               epoch=0, batch_idx=0, experiment=None):

      # Check model health before training step
      if not self.check_model_health(epoch, batch_idx):
          return float('inf'), float('inf'), float('inf')

      self.optimizer.zero_grad()

      # Enable gradients for S_T
      S_T = S_T.clone().detach().requires_grad_(True)

      try:
          pred_cashflow = self.model(portfolio, S_T)

          # Check prediction health
          if torch.isnan(pred_cashflow).any() or torch.isinf(pred_cashflow).any():
              print(f"🚨 Invalid predictions at Epoch {epoch}, Batch {batch_idx}")
              return float('inf'), float('inf'), float('inf')

          # Compute derivatives
          pred_deriv_from_autograd = None
          if true_derivative is not None:
              try:
                  pred_deriv_from_autograd = torch.autograd.grad(
                      outputs=pred_cashflow.sum(),
                      inputs=S_T,
                      retain_graph=True,
                      create_graph=True,
                      allow_unused=True
                  )[0]

                  if pred_deriv_from_autograd is not None:
                      if torch.isnan(pred_deriv_from_autograd).any() or torch.isinf(pred_deriv_from_autograd).any():
                          print(f"🚨 Invalid derivatives at Epoch {epoch}, Batch {batch_idx}")
                          pred_deriv_from_autograd = None

              except RuntimeError as e:
                  print(f"⚠️ Derivative computation failed: {e}")
                  pred_deriv_from_autograd = None

          total_loss, cashflow_loss, deriv_loss = self.compute_loss(
              pred_cashflow, cashflow, pred_deriv_from_autograd, true_derivative
          )

      except RuntimeError as e:
          print(f"⚠️ Forward pass failed at Epoch {epoch}, Batch {batch_idx}: {e}")
          return float('inf'), float('inf'), float('inf')

      if torch.isnan(total_loss) or torch.isinf(total_loss):
          print(f"⚠️ Invalid total loss at Epoch {epoch}, Batch {batch_idx}")
          return float('inf'), float('inf'), float('inf')

      # Backward pass
      scaled_loss = self.scaler.scale(total_loss)
      scaled_loss.backward()

      self.scaler.unscale_(self.optimizer)

      # Check gradients
      gradient_health = True
      max_grad_norm = 0.0
      for name, param in self.model.named_parameters():
          if param.grad is not None:
              grad_norm = param.grad.norm().item()
              max_grad_norm = max(max_grad_norm, grad_norm)

              if torch.isnan(param.grad).any() or torch.isinf(param.grad).any():
                  print(f"🚨 Bad gradient: {name} at Epoch {epoch}, Batch {batch_idx}")
                  gradient_health = False

      if not gradient_health or max_grad_norm > 50.0:
          print(f"🛑 Unhealthy gradients. Max norm: {max_grad_norm:.4f}")
          self.scaler.update()
          return float('inf'), float('inf'), float('inf')

      # Clip gradients
      total_norm_before = torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1e10)
      if total_norm_before > 5:
          total_norm_after = torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)
          print(f"Grad norm before: {total_norm_before:.2f}, after clipping: {total_norm_after:.2f}")

      # Log gradient stats
      if self.monitor_gradients and (epoch % 5 == 0 and batch_idx % 2000 == 0):
          total_norm, gradient_stats, _ = compute_gradient_stats(self.model)
          print_gradient_summary(gradient_stats, total_norm, epoch, batch_idx)


      # Optimizer step
      self.scaler.step(self.optimizer)
      self.scaler.update()
      self.scheduler.step()

      return total_loss.item(), cashflow_loss.item(), deriv_loss.item()



    def val_step(self, portfolio, S_T, cashflow, derivative):
        self.model.eval()
        S_T.requires_grad_(True)

        with torch.no_grad():
            with torch.enable_grad():
                pred_cashflow = self.model(portfolio, S_T)
                pred_deriv_from_autograd = torch.autograd.grad(
                    outputs=pred_cashflow,
                    inputs=S_T,
                    grad_outputs=torch.ones_like(pred_cashflow),
                    retain_graph=False,
                    create_graph=False
                )[0]

            val_total_loss, val_cashflow_loss, val_deriv_loss = self.compute_loss(
                pred_cashflow, cashflow, pred_deriv_from_autograd, derivative
            )

        if S_T.requires_grad:
            S_T.requires_grad_(False)

        return val_total_loss.item(), val_cashflow_loss.item(), val_deriv_loss.item()


    def update_scale(self, current_epoch):
        # ... (no changes needed here) ...
        if hasattr(self.model, 'branch_scale') and hasattr(self.model, 'trunk_scale'):
            # Linear warmup from initial_scale to final_scale over scale_warmup_epochs
            if current_epoch < self.scale_warmup_epochs:
                factor = (current_epoch + 1) / self.scale_warmup_epochs
                new_scale = self.initial_scale + (self.final_scale - self.initial_scale) * factor
            else:
                new_scale = self.final_scale

            with torch.no_grad():
                self.model.branch_scale.fill_(new_scale)
                self.model.trunk_scale.fill_(new_scale)

            print(f"🔧 [Epoch {current_epoch}] Updated branch/trunk scale → {new_scale:.4f}")


In [ ]:
experiment.end()

In [ ]:
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
hidden_dim = 64
latent_dim = 64
batch_size = 128

portfolio_feature_dim = 3

PORT_LEN = 50
PORT_SAMPLE_SIZE = 10000
FEED_ST_LEN_EACH_PORT = 150

final_save_path = '/content/drive/MyDrive/Ucl/'
model_path = final_save_path + 'final_deeponet_model.pt'

raw_dataset = OperatorDatasetStandarization(
      num_samples=PORT_SAMPLE_SIZE,
      portfolio_size=PORT_LEN,
      num_samples_S_T=FEED_ST_LEN_EACH_PORT,
      is_fitting_mode=True  # No normalization during fitting
      )

raw_data_loader = DataLoader(raw_dataset, batch_size=batch_size, shuffle=False)

K_scalar, S_T_scalar = fit_K_ST_scalers(raw_data_loader)
cashflow_scaler = fit_cashflow_scaler(raw_data_loader)

normalized_dataset = OperatorDatasetStandarization(
        num_samples=PORT_SAMPLE_SIZE,
        portfolio_size=PORT_LEN,
        num_samples_S_T=FEED_ST_LEN_EACH_PORT,
        K_Scalar=K_scalar,
        S_T_scalar=S_T_scalar,
        cashflow_scaler=cashflow_scaler,
        is_fitting_mode=False)

# --- Load the trained model ---
deeponet_model = OptimizedDeepONet(
    portfolio_feature_dim=portfolio_feature_dim,
    hidden_dim=hidden_dim,
    latent_dim=latent_dim,
    use_enhanced_transformer= True
).to(DEVICE)

try:
    deeponet_model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    print(f"Successfully loaded model from {model_path}")
except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}. Please ensure the model is trained and saved.")
    exit()
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

deeponet_model.eval()  # Set model to evaluation mode

eval_dataloader = DataLoader(normalized_dataset, batch_size=1, shuffle=False)

# Get a sample for plotting
with torch.no_grad():
    for i, (portfolio, S_T_i_normalized, true_cashflows_normalized, derivative) in enumerate(eval_dataloader):

        # Make prediction
        predicted_cashflows_normalized = deeponet_model(portfolio, S_T_i_normalized)

        # Squeeze the batch dimension
        predicted_cashflows_normalized = predicted_cashflows_normalized.squeeze(0)
        true_cashflows_normalized = true_cashflows_normalized.squeeze(0)
        S_T_i_normalized = S_T_i_normalized.squeeze(0)

        # 1. CHECK LOSS CALCULATION IN NORMALIZED SPACE
        mse_normalized = torch.mean((true_cashflows_normalized - predicted_cashflows_normalized) ** 2)

        # 2. ANALYZE NORMALIZED VALUES
        # Convert to numpy for scaler operations
        S_T_i_numpy = S_T_i_normalized.cpu().numpy()
        true_cashflows_numpy = true_cashflows_normalized.cpu().numpy()
        predicted_cashflows_numpy = predicted_cashflows_normalized.cpu().numpy()

        # 3. DENORMALIZE
        s_t_values_denormalized = S_T_scalar.inverse_transform(
            S_T_i_numpy.reshape(-1, 1)
        ).squeeze()

        true_cashflows_denormalized = cashflow_scaler.inverse_transform(
            true_cashflows_numpy.reshape(-1, 1)
        ).squeeze()

        predicted_cashflows_denormalized = cashflow_scaler.inverse_transform(
            predicted_cashflows_numpy.reshape(-1, 1)
        ).squeeze()

        # 4. DENORMALIZED ERROR ANALYSIS
        mse_denormalized = mean_squared_error(true_cashflows_denormalized, predicted_cashflows_denormalized)
        rmse_denormalized = np.sqrt(mse_denormalized)

        # 5. SORT DATA FOR PLOTTING
        # For normalized plots
        sorted_indices_norm = np.argsort(S_T_i_numpy)
        s_t_values_norm_sorted = S_T_i_numpy[sorted_indices_norm]
        true_cashflows_norm_sorted = true_cashflows_numpy[sorted_indices_norm]
        predicted_cashflows_norm_sorted = predicted_cashflows_numpy[sorted_indices_norm]

        # For denormalized plots
        sorted_indices = np.argsort(s_t_values_denormalized)
        s_t_values_sorted = s_t_values_denormalized[sorted_indices]
        true_cashflows_sorted = true_cashflows_denormalized[sorted_indices]
        predicted_cashflows_sorted = predicted_cashflows_denormalized[sorted_indices]

        # 6. COMPREHENSIVE PLOTTING - NOW WITH NORMALIZED AND DENORMALIZED
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))

        # ===== NORMALIZED SPACE PLOTS =====
        # Plot 1: Normalized comparison
        axes[0, 0].plot(s_t_values_norm_sorted, true_cashflows_norm_sorted,
                       label='True', color='blue', marker='o', markersize=3, linewidth=2)
        axes[0, 0].plot(s_t_values_norm_sorted, predicted_cashflows_norm_sorted,
                       label='Predicted', color='red', linestyle='--', marker='x', markersize=3, linewidth=2)
        axes[0, 0].set_xlabel('S_T (Normalized)')
        axes[0, 0].set_ylabel('Cashflow (Normalized)')
        axes[0, 0].set_title(f'Cashflow Prediction\nRMSE: {rmse_denormalized:.6f}')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Plot 2: Normalized residuals
        residuals_norm = true_cashflows_norm_sorted - predicted_cashflows_norm_sorted
        axes[1, 0].scatter(s_t_values_norm_sorted, residuals_norm, alpha=0.6, color='green', s=20)
        axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[1, 0].set_xlabel('S_T (Normalized)')
        axes[1, 0].set_ylabel('Residuals (Normalized)')
        axes[1, 0].set_title(f'Cashflow Residuals\nMAE: {np.mean(np.abs(residuals_norm)):.6f}')
        axes[1, 0].grid(True, alpha=0.3)

        # ===== DENORMALIZED SPACE PLOTS =====
        # Plot 3: Denormalized comparison
        axes[0, 1].plot(s_t_values_sorted, true_cashflows_sorted,
                       label='True', color='blue', marker='o', markersize=3, linewidth=2)
        axes[0, 1].plot(s_t_values_sorted, predicted_cashflows_sorted,
                       label='Predicted', color='red', linestyle='--', marker='x', markersize=3, linewidth=2)
        axes[0, 1].set_xlabel('S_T (Denormalized)')
        axes[0, 1].set_ylabel('Cashflow (Denormalized)')
        axes[0, 1].set_title(f'Cashflow Prediction Comparison\nRMSE: {rmse_denormalized:.6f}')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Plot 4: Denormalized residuals
        residuals = true_cashflows_sorted - predicted_cashflows_sorted
        axes[1, 1].scatter(s_t_values_sorted, residuals, alpha=0.6, color='green', s=20)
        axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[1, 1].set_xlabel('S_T (Denormalized)')
        axes[1, 1].set_ylabel('Residuals (Denormalized)')
        axes[1, 1].set_title(f'Cashflow Residuals\nMAE: {np.mean(np.abs(residuals)):.6f}')
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        # Break after first sample
        if i == 0:
            break
